# Protein infenrece
We can find the most likely observed protein in a protein group by making an estimated guess using the annonation data from uniprot. By looking at reviewed status, annotation score, number of entry and enty name lenth (newer entries have longer names) we can find the most relevant protein. To compare search results, we then compare the gene names. 

Download the reference proteome from uniprot in tsv format (https://www.uniprot.org/proteomes/UP000005640) and select the data for gene names (primatry), annotation, go terms and whatever extra information you'd like to look at. Remeber, all isoforms are under the main entry for each protein. 

In [1]:
import re
import os
import pandas as pd
import numpy as np

In [30]:
annos = "PATH_TO_ANNOTATIONS_FILE/uniprotkb_proteome_UP000005640_2025_03_18.tsv.gz"

f_path = "PATH_TO_FASTA_FILE/HUMAN.fasta"

IN_PATH = "PATH/proteinGroups.txt"
OUT_PATH = "PATH/proteinGroups_w_uniprot.txt"

# Protein inference 

In [31]:
uni_annos = pd.read_csv(annos, delimiter="\t")
uni_annos['Gene Ontology IDs'] = uni_annos['Gene Ontology IDs'].fillna('')

def count_go_and_keywords(row):
    go_terms_count = len(row['Gene Ontology IDs'].split(";")) if row['Gene Ontology IDs'] else 0
    keywords_count = len(row['Keywords'].split(";")) if row['Keywords'] else 0
    return go_terms_count + keywords_count

# Apply the function and add a new column with count of go terms
uni_annos['Total Count'] = uni_annos.apply(count_go_and_keywords, axis=1)
uni_annos['Entry name len'] = uni_annos['Entry'].str.len()

In [20]:
def find_uni(df, annos): 
    
    for index, row in df.iterrows(): 
    
        test_proteins = row['Proteins_list']
        filtered_df = annos[annos['Entry'].isin(test_proteins)]

        sorted_df = filtered_df.sort_values(by=['Reviewed', 'Total Count', 'Annotation', 'Entry', 'Entry name len'], ascending=[True, False, False, True, False])
        sorted_df = sorted_df.reset_index(drop=True)

        if not sorted_df.empty: 

            # Define output columns
            df.at[index, 'Uniprot_cannonised'] = sorted_df['Entry'][0] # main extracted uniprot id
            df.at[index, 'Gene_name'] = sorted_df['Gene Names (primary)'][0] # main extracted gene name
            
    return df

In [22]:
def process_data_msms_MQ(datafile, uni_annos): 
    '''''''
    data = pd.read_csv(datafile, delimiter="\t")
    data['Proteins_list'] = data['Proteins'].apply(lambda x: x.split(';') if pd.notnull(x) else [])
    data['Proteins_list'] = data['Proteins_list'].apply(lambda sublist: [item.split('-')[0] if '-' in item else item for item in sublist])
    data['Uniprot_cannonised'] = "" 
    data['Gene_name'] = ""

    data_cannonised = find_uni(data, uni_annos)
    
    return data_cannonised

In [23]:
def process_pg_MQ(datafile, uni_annos):
    data = pd.read_csv(datafile, delimiter="\t")
    data['Proteins_list'] = data['Protein IDs'].apply(lambda x: x.split(';') if pd.notnull(x) else [])
    data['Proteins_list'] = data['Proteins_list'].apply(lambda sublist: [item.split('|')[1] if '|' in item else item for item in sublist])
    data['Uniprot_cannonised'] = "" 
    data['Gene_name'] = ""

    data_cannonised = find_uni(data, uni_annos)
    
    return data_cannonised


In [ ]:
data_processed = process_data_msms_MQ(IN_PATH, uni_annos)
# save to tsv file
data_processed.to_csv(OUT_PATH, index=False, sep = '\t')